In [1]:
import numpy as np
import cv2 

In [2]:
# Parameters for ShiTomasi corner detection (good features to track paper)
corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

In [3]:
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (200,200),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [4]:
# Capture the video
cap = cv2.VideoCapture(0)

# Grab the very first frame of the stream
ret, prev_frame = cap.read()

# Grab a grayscale image (We will refer to this as the previous frame)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Grabbing the corners
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Create a matching mask of the previous frame for drawing on later
mask = np.zeros_like(prev_frame)



while True:
    
    # Grab current frame
    ret,frame = cap.read()
    
    # Grab gray scale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Optical Flow on the Gray Scale Frame
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    
    # Using the returned status array (the status output)
    # status output status vector (of unsigned chars); each element of the vector is set to 1 if
    # the flow for the corresponding features has been found, otherwise, it is set to 0.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]
    
    # Use ravel to get points to draw lines and circles
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        print(x_new,y_new, x_prev, y_prev)
        xx_new=int(x_new)
        yy_new=int(y_new)
        xx_prev=int(x_prev)
        yy_prev=int(y_prev)
        
        # Lines will be drawn using the mask created from the first frame
        mask=cv2.line(mask, (xx_new,yy_new),(xx_prev,yy_prev),(0,255,0),3)
    
        
        # Draw red circles at corner points
        frame = cv2.circle(frame, (xx_new,yy_new),8,(0,0,255),-1)
    
    # Display the image along with the mask we drew the line on.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
   
    # Now update the previous frame and previous points
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)
    
    
cv2.destroyAllWindows()
cap.release()

271.002 275.79138 271.0 276.0
562.36224 311.55597 563.0 312.0
55.898354 359.81708 56.0 360.0
262.00128 256.83932 262.0 257.0
281.01898 298.7962 281.0 299.0
270.96228 275.46906 271.002 275.79138
561.23035 311.05835 562.36224 311.55597
55.829678 359.6573 55.898354 359.81708
261.95477 256.54477 262.00128 256.83932
280.98245 298.4193 281.01898 298.7962
270.91403 275.65656 270.96228 275.46906
560.5457 312.23373 561.23035 311.05835
55.926323 359.8837 55.829678 359.6573
261.91107 256.7389 261.95477 256.54477
280.94177 298.6572 280.98245 298.4193
270.94373 275.38852 270.91403 275.65656
560.0718 312.61914 560.5457 312.23373
55.859005 359.79593 55.926323 359.8837
261.93445 256.43945 261.91107 256.7389
280.98102 298.37805 280.94177 298.6572
271.04767 275.4059 270.94373 275.38852
560.05145 312.8767 560.0718 312.61914
55.912994 359.86108 55.859005 359.79593
262.04285 256.46155 261.93445 256.43945
281.08316 298.39734 280.98102 298.37805
271.16736 275.37274 271.04767 275.4059
560.6688 312.80707 560.0

In [ ]:
mask